## Заголовки новин

### 1. Форматування

[The Associated Press Stylebook](https://www.amazon.com/Associated-Press-Stylebook-2017-Briefing/dp/0465093043/) - це посібник зі стилю, яким часто послуговуються журналісти по всьому світу. Він рекомендує такі правила форматування заголовків:
1. З великої літери потрібно писати іменники, займенники, дієслова, прикметники, прислівники та підрядні сполучники. Якщо слово написане через дефіс, велику літеру потрібно додати для кожної частинки слова (наприклад, правильно "Self-Reflection", а не "Self-reflection").
2. З великої літери потрібно писати перше і останнє слово заголовку, незалежно від частини мови.
3. З маленької літери потрібно писати всі інші частини мови: артиклі/визначники, сурядні сполучники, прийменники, частки, вигуки.

**Завдання:**
1. напишіть програму, яка форматує заголовки за вказаними правилами
2. проженіть вашу програму на [корпусі заголовків з The Examiner](examiner-headlines.txt)
3. збережіть програму та файл із відформатованими заголовками у директорії з вашим іменем
4. скільки заголовків у корпусі було відформатовано правильно? (скільки заголовків залишились незмінними?)

Зверніть увагу, що ваша програма повинна правильно розрізняти прийменники та підрядні сполучники. Наприклад, `Do as you want` => `Do As You Want` (бо "as" тут є сполучником), but `How to use a Macbook as a table` => `How to Use a Macbook as a Table` (бо "as" тут є прийменником).

In [8]:
import spacy
nlp = spacy.load('en')

def style_capitalize(token):
    if token.isupper():
        return token
    elif '-' in token:
        parts = token.split('-')
        capit_parts = [p.capitalize() if not p.isupper() else p for p in parts]
        return '-'.join(capit_parts)
    else:
        return token.capitalize()

def join_tokens(token_idx_list):
    string = ''
    prev_idx = 0
    prev_token_len = 0
    for  token, idx in token_idx_list:
        string += ' '*(idx - prev_idx - prev_token_len) + token
        prev_idx = idx
        prev_token_len = len(token)
        
    return string
    
def press_style(string):
    tokens = nlp(string)
    
    if tokens[-1].text == '\n':
        tokens = tokens[:-1]
        
    capit_tokens = []
    for token in tokens:
        if (token.i == 0) or (token.i == len(tokens) - 1):
            capit_tokens.append((style_capitalize(token.text), token.idx))
        elif token.pos_ in set(['NOUN', 'PROPN', 'PRON', 'VERB', 'ADJ', 'ADV']):
            capit_tokens.append((style_capitalize(token.text), token.idx))
        elif token.pos_ == 'ADP' and token.dep_ == 'mark':
            capit_tokens.append((style_capitalize(token.text), token.idx))                   
        else:
            capit_tokens.append((token.text, token.idx))
    
    return join_tokens(capit_tokens) + '\n'

input_file = open("examiner-headlines.txt", "r")
lines = input_file.readlines()

5000

In [10]:
output_file = open("output.txt", "w+")
with open("output.txt", 'a') as f:
    for line in lines:
        f.write(press_style(line))

### 2. Вірусні новини

У статті [Automatic Extraction of News Values from Headline Text](http://www.aclweb.org/anthology/E17-4007) описано основні ознаки заголовків, які кидаються в очі і змушують читача таки прочитати новину:
1. наявність імен людей, назв компаній тощо
2. емоційне забарвлення
3. ступені порівняння
4. близькість
5. елемент несподіванки
6. унікальність

**Завдання:**
1. Напишіть програму, яка аналізує заголовок за першими трьома ознаками (у спрощеній формі)
   * Чи є в заголовку іменовані стуності?
   * Чи є заголовок позитивно чи негативно забарвлений?
   * Чи є в заголовку прикметники та прислівники вищого і найвищого ступенів порівняння?
2. Проженіть вашу програму на [корпусі заголовків з The Examiner](examiner-headlines.txt). Для кожної з трьох ознак, визначте відсоток заголовків у корпусі, які її мають.
3. Збережіть програму та пораховану статистику в директорії з вашим іменем.

Додаткова інформація:
- Типи сутностей, які впливають на "вірусність" заголовка, виберіть самостійно.
- Для визначення емоційного забарвлення, використайте [SentiWordNet](http://sentiwordnet.isti.cnr.it/). Наприклад, можна перевірити, що середнє значення позитивності/негативності слова у заголовку перевищує 0.5. Для визначення середнього значення можна брати до п'яти перших значень слова з такою частиною мови. Будьте креативними та експериментуйте.

### Джерела

Ви можете використати будь-яку мову програмування та будь-яку NLP-бібліотеку.

Набір заголовків взятий із https://www.kaggle.com/therohk/examine-the-examiner.

In [182]:
import nltk
from nltk.corpus import sentiwordnet as swn #, SentiSynset
import numpy as np
from collections import namedtuple

def spacy_pos_to_wn(pos):
    if pos in set(['NOUN', 'PROPN']):
        return 'n'
    elif pos.startswith('ADJ'):
        return 'a'
    elif pos.startswith('VERB'):
        return 'v'
    elif pos.startswith('ADV'):
        return 'r'
    return None

def spam_title(string):    
    tokens = nlp(string)
    
    if tokens[-1].text == '\n':
        tokens = tokens[:-1]
    
    sentiments = []
    named_ent_bool = False
    comp_superlative_bool = False
    target_tags=['JJR', 'JJS', 'RBR', 'RBS']
    
    for token in tokens:
        # Named Entities
        if token.ent_type_ in set(['PERSON', 'ORG', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'DATE', 'MONEY']):
            named_ent_bool = True
        
        # Sentiment
        wn_pos = spacy_pos_to_wn(token.pos_)
        if wn_pos:
            syns = list(swn.senti_synsets(token.text, pos=wn_pos)) 
            if syns:
                # Calculate word sentiment as a mean value of the first 3 sentiments
                syns_sentiments = [s.pos_score() - s.neg_score() for s in syns[0:4]]
                sentiment = np.asarray(syns_sentiments).mean()
                sentiments.append(sentiment)
           
        # Comparative / Superlative
        if token.tag_ in target_tags:
            comp_superlative_bool = True
    
    named_ents = int(named_ent_bool)
    
    
    sentiment_num = np.asarray(sentiments).mean()
    sentiment = 0 if abs(sentiment_num) <= 0.5 else 1
    
    comp_superlative = int(comp_superlative_bool)
    
    return named_ents, sentiment, comp_superlative
    

In [183]:
spam_title('Halep slow enters Rogers Cup final in straight sets win over Errani')

(1, 0, 0)

In [184]:
input_file = open("examiner-headlines.txt", "r")
lines = input_file.readlines()

titles_spam_analysis = []
for l in lines:
    named_ents, sentiment, comp_superlative = spam_title(l)
    titles_spam_analysis.append([named_ents, sentiment, comp_superlative])

In [185]:
np_all = np.asarray(titles_spam_analysis)
np_stat = np_all.sum(axis=0) / np_all.shape[0]
np_stat

array([0.6886, 0.0028, 0.0448])

In [186]:
output_file = open("spam_output.txt", "w+")
with open("spam_output.txt", 'a') as f:
    f.write('named_ents, sentiment, comp_superlative\n')
    f.write(', '.join([str(i) for i in list(np_stat)]))